**Packages**

In [ ]:
! pip install datasets --quiet
! pip install evaluate --quiet
! pip install rouge_score --quiet
! pip install sacrebleu --quiet
! pip install transformers --quiet
! pip install -q sentencepiece --quiet
! pip install summarizer --quiet
! pip install bert-extractive-summarizer --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 63.2 MB/s eta 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import math

from datasets import load_dataset
import evaluate

import inspect

#let's make longer output readable without horizontal scrolling
from pprint import pprint

import warnings

import regex as re

import os, re
import time

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# These auto classes load the right type of tokenizer and model based on a model name
from transformers import AutoTokenizer, TFAutoModel
from transformers import pipeline
from transformers import AutoModel

**Necessary Functions**

In [ ]:
rouge = evaluate.load('rouge')

In [ ]:
chrf = evaluate.load("chrf")

In [ ]:
def get_default_args(func):
    signature = inspect.signature(func)
    return {
        k: v.default
        for k, v in signature.parameters.items()
        if v.default is not inspect.Parameter.empty
    }

**Huggingface Transformers Training Resources**

https://github.com/huggingface/transformers/blob/main/examples/pytorch/summarization/run_summarization.py

https://www.databricks.com/blog/2023/03/20/fine-tuning-large-language-models-hugging-face-and-deepspeed.html

https://gitlab.com/nicolalandro/summarization

Load Data

In [ ]:
df = pd.read_csv('../Data/xl_sum_sample_test.csv')

In [ ]:
df_pronouns = pd.read_csv('../Data/xl_sum_sample_test_pronouns.csv')

In [ ]:
def generate_scores(mod, data, do_sample = False, num_beams = 1, top_k = 50, num_beam_groups = 1):

    targets = []
    candidates = []
    r1 = []
    r2 = []
    rL = []
    rLs = []
    chrfs = []

    for i in range(int(len(data['text']))):

        candidate = mod(data['text'][i], 
                              truncation = True,
                              max_length = 256,
                              min_length = 0, 
                              do_sample = do_sample,
                              num_beams = num_beams, 
                              top_k = top_k,
                              num_beam_groups = num_beam_groups,
                              )[0]
        candidate = [candidate['summary_text']]

        ref = [data['summary'][i]]

        results = rouge.compute(predictions=candidate,
                              references=ref)

        targets.append(ref)
        candidates.append(candidate)
        r1.append(results['rouge1'])
        r2.append(results['rouge2'])
        rL.append(results['rougeL'])
        rLs.append(results['rougeLsum'])

        results = chrf.compute(predictions=candidate,
                              references=ref)

        chrfs.append(results['score'])

    return pd.DataFrame({'target': targets, 'candidate': candidates,
                  'rouge1': r1, 'rouge2': r2, 'rougeL': rL, 
                      'rougeLs': rLs, 'chrf': chrfs})

# Baseline

In [ ]:
def baseline_scores(data):
    base_r1 = []
    base_r2 = []
    base_rL = []
    base_rLs = []
    base_chrf = []
    candidates = []
    targets = []
    for i in range(len(data['text'])): 


        # first three sentences 
        candidate = ". ".join(data["text"][i].split('. ')[0:3]) + "."
        candidate = [candidate]

        ref = [data['summary'][i]]

        results = rouge.compute(predictions=candidate,
                              references= ref)

        results2 = chrf.compute(predictions=candidate,
                              references= ref)

        candidates.append(candidate)
        targets.append(ref)
        base_r1.append(results['rouge1'])
        base_r2.append(results['rouge2'])
        base_rL.append(results['rougeL'])
        base_rLs.append(results['rougeLsum'])

        base_chrf.append(results2['score'])

    return pd.DataFrame({'target': targets, 'candidate': candidates,
                'rouge1': base_r1, 'rouge2': base_r2, 'rougeL': base_rL, 
                    'rougeLs': base_rLs, 'chrf': base_chrf})

*All Categories*

In [ ]:
baseline_score_df = baseline_scores(df)

In [ ]:
baseline_score_df.to_csv('baseline_all_categories.csv')

*Proper Nouns*

In [ ]:
baseline_pronouns = baseline_scores(df_pronouns)

In [ ]:
baseline_pronouns.to_csv('baseline_pronouns.csv')

# T5 SCORES

### Base

In [ ]:
t5_base_summarizer = pipeline("summarization", model="t5-base")

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


*All Categories*

In [ ]:
t5_base_scores_df = generate_scores(t5_base_summarizer, df, do_sample = False, num_beams = 4, top_k = 75, num_beam_groups = 2)

In [ ]:
t5_base_scores_df.to_csv('t5_all_categories_base.csv')

*Proper Nouns*

In [ ]:
t5_base_pronouns_scores_df = generate_scores(t5_base_summarizer, df_pronouns, do_sample = False, num_beams = 4, top_k = 75, num_beam_groups = 2)

Your max_length is set to 256, but you input_length is only 204. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=102)
Your max_length is set to 256, but you input_length is only 192. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=96)
Your max_length is set to 256, but you input_length is only 240. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=120)


In [ ]:
t5_base_pronouns_scores_df.to_csv('t5_pronouns_base.csv')

### Finetuned

In [ ]:
t5_finetuned_summarizer = pipeline("summarization", model="arisanguyen/finetuned_T5_all_categories", revision = 'model_0')

*All Categories*

In [ ]:
t5_scores_df = generate_scores(t5_finetuned_summarizer, df, do_sample = False, num_beams = 4, top_k = 75, num_beam_groups = 2)

In [ ]:
t5_scores_df.to_csv('t5_all_categories_finetuned.csv')

*Proper Nouns*

In [ ]:
t5_prounouns_scores_df = generate_scores(t5_finetuned_summarizer, df_pronouns, do_sample = False, num_beams = 4, top_k = 75, num_beam_groups = 2)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Your max_length is set to 256, but you input_length is only 204. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=102)
Your max_length is set to 256, but you input_length is only 192. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=96)
Your max_length is set to 256, but you input_length is only 240. You might consider decreasing max_length manually, e.g. summarizer('...', max_lengt

In [ ]:
t5_prounouns_scores_df.to_csv('t5_pronouns_finetuned.csv')

# BART SCORES

### Base

In [ ]:
def generate_bart_scores(model, data, do_sample = False, num_beams = 1, top_k = 50):

    targets = []
    candidates = []
    r1 = []
    r2 = []
    rL = []
    rLs = []
    chrfs = []

    for i in range(int(len(data['text']))):
        candidate = model(data['text'][i], 
                               truncation = True, 
                               max_length = 256, 
                               min_length = 0, 
                               do_sample = do_sample,
                               num_beams = num_beams, 
                               top_k = top_k,
                                )[0]
        candidate = [candidate['summary_text']]

        ref = [data['summary'][i]]

        results = rouge.compute(predictions=candidate,
                              references=ref)

        targets.append(ref)
        candidates.append(candidate)
        r1.append(results['rouge1'])
        r2.append(results['rouge2'])
        rL.append(results['rougeL'])
        rLs.append(results['rougeLsum'])

        results = chrf.compute(predictions=candidate,
                              references=ref)

        chrfs.append(results['score'])

    return pd.DataFrame({'target': targets, 'candidate': candidates,
                  'rouge1': r1, 'rouge2': r2, 'rougeL': rL, 
                      'rougeLs': rLs, 'chrf': chrfs})

In [ ]:
bart_base_summarizer = pipeline("summarization", model="facebook/bart-base")

*All Categories*

In [ ]:
bart_base_scores_df = generate_bart_scores(bart_base_summarizer, df, do_sample = True, num_beams = 4, top_k = 75)

Your max_length is set to 128, but you input_length is only 86. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=43)
Your max_length is set to 128, but you input_length is only 92. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=46)
Your max_length is set to 128, but you input_length is only 70. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=35)


In [ ]:
bart_base_scores_df.to_csv('bart_all_categories_base.csv')

*Proper Nouns*

In [ ]:
bart_base_pronouns_scores_df = generate_bart_scores(bart_base_summarizer, df_pronouns, do_sample = True, num_beams = 4, top_k = 75, num_beam_groups = 2)

In [ ]:
bart_base_pronouns_scores_df.to_csv('bart_pronouns_base.csv')

### Finetuned

In [ ]:
bart_finetuned_summarizer = pipeline("summarization", model="arisanguyen/finetuned-BART-all-categories", revision = 'model_2')

*All Categories*

In [ ]:
bart_scores_df = generate_scores(bart_finetuned_summarizer, df, do_sample = True, num_beams = 4, top_k = 75, num_beam_groups = 2)

In [ ]:
bart_scores_df.to_csv('bart_all_categories_finetuned.csv')

*Proper Nouns*

In [ ]:
bart_pronouns_scores_df = generate_scores(bart_finetuned_summarizer, df_pronouns, do_sample = False, num_beams = 4, top_k = 75, num_beam_groups = 2)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Your max_length is set to 256, but you input_length is only 187. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=93)
Your max_length is set to 256, but you input_length is only 175. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=87)
Your max_length is set to 256, but you input_length is only 217. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=108)
Your max_length is set to 256, but you input_length is only 244. You might consider decreasing max_length manually, e.g. summarizer('..

In [ ]:
bart_pronouns_scores_df.to_csv('bart_pronouns_finetuned.csv')

# BERT SCORES

In [ ]:
def generate_bert_scores(model, data, do_sample = False, num_beams = 1, top_k = 50, num_beam_groups = 1):

        targets = []
        candidates = []
        r1 = []
        r2 = []
        rL = []
        rLs = []
        chrfs = []

        for i in range(int(len(data['text']))):

        candidate = model(data['text'][i], 
                      num_sentences = 2, 
                      min_length = 0,
                      max_length = 256,        
                      ratio = None,
                      use_first = False,
                     )
        candidate = [candidate]

        ref = [data['summary'][i]]

        results = rouge.compute(predictions=candidate,
                              references=ref)

        targets.append(ref)
        candidates.append(candidate)
        r1.append(results['rouge1'])
        r2.append(results['rouge2'])
        rL.append(results['rougeL'])
        rLs.append(results['rougeLsum'])

        results = chrf.compute(predictions=candidate,
                              references=ref)

        chrfs.append(results['score'])
  
    return pd.DataFrame({'target': targets, 'candidate': candidates,
                  'rouge1': r1, 'rouge2': r2, 'rogueL': rL, 
                      'rogueLs': rLs, 'chrf': chrfs})

In [ ]:
from summarizer import Summarizer
bert_model = Summarizer('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


*All Categories*

In [ ]:
bert_scores_df = generate_bert_scores(bert_model, df, do_sample = False, num_beams = 4, top_k = 75, num_beam_groups = 2)

In [ ]:
bert_scores_df.to_csv('bert_all_categories.csv')

*Proper Nouns*

In [ ]:
bert_pronouns_scores_df = generate_bert_scores(bert_model, df_pronouns, do_sample = True, num_beams = 4, top_k = 75, num_beam_groups = 2)

In [ ]:
bert_pronouns_scores_df.to_csv('bert_pronouns.csv')